In [1]:
from xlcalibur import dataclasses, housekeeping, dataaccess 
from xlcalibur.xcom import Packets
from xlcalibur.systems import Systems
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import os

/grad/ephraimgau/xlcalflightsoftware/source/core/src/core/XLoggingSystem.cpp:377: DEBUG: Reading logging configuration from /grad/ephraimgau/xlcalflightsoftware/install-debug/share/xlcalibur/conf/logging.conf
/grad/ephraimgau/xlcalflightsoftware/source/core/src/core/XLoggingSystem.cpp:377: DEBUG: Reading logging configuration from logging.conf


In [2]:
def analyze_file(n):
    filepath = f"/data/xlcal/data/2024_convData-P-ns/Run{n:06d}.dat"
    print(f"read file Run{n:06d}.dat")

    infile = dataclasses.XDataFile(filepath)

    all_alignment = [p.Deserialize() for p in infile.Scan(Packets.X_PKT_ALIGNMENT_DATA)]
    
    good_alignment = [p for p in all_alignment if p.fit_valid]
    valid_alignment = [p for p in good_alignment if np.isfinite(p.center_x) and np.isfinite(p.center_y) and p.scale != 0]

    if not valid_alignment:
        print(f" Run{n:06d} empty, skip")
        return None

    meanx = np.average([a.center_x for a in valid_alignment])
    meany = np.average([a.center_y for a in valid_alignment])
    t0 = valid_alignment[0].time.to_mjd()
    tdata = [86400 * (a.time.to_mjd() - t0) for a in valid_alignment]
    xdata = [(a.center_x - meanx) / a.scale for a in valid_alignment]
    ydata = [(a.center_y - meany) / a.scale for a in valid_alignment]
    rotation_data = [a.angle * 180 / np.pi for a in valid_alignment]
    mjd_times = np.array(tdata) / 86400 + t0

    case1, case2, case3 = [], [], []

    for i in range(len(xdata)):
        x = xdata[i]
        y = ydata[i]
        t = rotation_data[i]
        mjd = mjd_times[i]

        if abs(x) < 2 and abs(y) < 2:
            case1.append(mjd)
        elif x > 2 and abs(y) < 2:
            case2.append(mjd)
        elif x < -2 and abs(y) < 2:
            case3.append(mjd)

    return case1, case2, case3

In [3]:
def save_all_runs(run_list):
    all_case1 = []
    all_case2 = []
    all_case3 = []

    for n in run_list:
        result = analyze_file(n)
        if result:
            c1, c2, c3 = result
            all_case1.extend(c1)
            all_case2.extend(c2)
            all_case3.extend(c3)

    maxlen = max(len(all_case1), len(all_case2), len(all_case3))
    df = pd.DataFrame({
        "Case 1 (|X|,|Y|<2mm)": all_case1 + [None]*(maxlen - len(all_case1)),
        "Case 2 (X>2mm)": all_case2 + [None]*(maxlen - len(all_case2)),
        "Case 3 (X<-2mm)": all_case3 + [None]*(maxlen - len(all_case3)),
    })

    os.makedirs("output", exist_ok=True)
    df.to_csv("output/summary_all_mjd.csv", index=False)
    print("saved output/summary_all_mjd.csv")


In [ ]:
if __name__ == "__main__":
    runs = range(13630,13671)
    save_all_runs(runs)